### 1. Prep the city data

From yesterday's work we copy the final webscraping function and call it.

In [ ]:
!pip install lat_lon_parser

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from lat_lon_parser import parse    # for decimal coordinates


def cities_dataframe(cities):

  city_data = []

  for city in cities:
    url = f"https://www.wikipedia.org/wiki/{city}"
    response = requests.get(url)
    city_soup = BeautifulSoup(response.content, 'html.parser')

    # extract the relevant information
    city_latitude = city_soup.find(class_="latitude").get_text()
    city_longitude = city_soup.find(class_="longitude").get_text()
    country = city_soup.find(class_="infobox-data").get_text()

    # keep track of data per city
    city_data.append({"City": city,
                    "Country": country,
                    "Latitude": parse(city_latitude), # latitude in decimal format
                    "Longitude": parse(city_longitude), # longitude in decimal format
                    })

  return pd.DataFrame(city_data)


In [ ]:
# call the function
list_of_cities = ["Berlin", "Hamburg", "Munich"]

cities_df = cities_dataframe(list_of_cities)
cities_df

,City,Country,Latitude,Longitude
0,Berlin,Germany,52.5200,13.405
1,Hamburg,Germany,53.5500,10.000
2,Munich,Germany,48.1375,11.575


### 2. Prep the SQL cities table

Create a schema and table to hold the data in the above DataFrame. Notice that the columns all have the same name, in the same order, as the DataFrame.

The following code would be run on MySQL Workbench.

```sql
DROP SCHEMA IF EXISTS gans;
CREATE SCHEMA gans;
USE gans;

CREATE TABLE cities (
  `City_ID` INT UNSIGNED NOT NULL AUTO_INCREMENT, -- for primary key
  `City` VARCHAR(80) NOT NULL,
  `Country` VARCHAR(80) NOT NULL,
  `Latitude` FLOAT NOT NULL,
  `Longitude` FLOAT NOT NULL,
  PRIMARY KEY (`ID`)
  );
  
  ```

### 3. Push cities data to SQL database

In [ ]:
schema = "gans"  # as in the CREATE SCHEMA statement above
host = "127.0.0.1" # "localhost" also works
user = "root"  # or your username if you set up a different one
password = "YOURMYSQLPASSWORD"
port = 3306  # standard; don't change

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [ ]:
cities_df.to_sql("cities",
                 if_exists="append",
                 con=connection_string,
                 index=False)

3

If successful, the above code block will return a 3 (the number of rows in the DataFrame).

### 4. Pull cities from SQL database

The cities were assigned Primary Key information when they were sent to the SQL database (the auto-incrementing ID column), so we need to fetch that data to get the keys.

In [ ]:
cities_df = pd.read_sql("cities", con=connection_string)

cities_df

,City_ID,City,Country,Latitude,Longitude
0,1,Berlin,Germany,52.5200,13.405
1,2,Hamburg,Germany,53.5500,10.000
2,3,Munich,Germany,48.1375,11.575


### 5. Prep and push the population data

Now that we have cities with IDs, we can modify yesterday's function to gather population information so that those new IDs appear in the DataFrame as well.

Since we need two pieces of information from each row of the DataFrame (ID and City), we use `.iterrows()` to iterate row by row.

In [ ]:
from datetime import datetime # to get today's date

# create one single list to keep track of all values (instead of having separate lists)
population_data = []

for i, row in cities_df.iterrows():  # i is the row's index; row is the Series containing that row's data
    url = f"https://www.wikipedia.org/wiki/{row['City']}"
    response = requests.get(url)
    city_soup = BeautifulSoup(response.content, 'html.parser')

    # extract the relevant information
    city_population = city_soup.find(string="Population").find_next("td").get_text()
    city_population_clean = city_population.replace(",", "")
    today = datetime.today().date()  # modified to a date object to fit into MySQL DATE datatype

    # for each city we append a dictionary of values to the list
    population_data.append({"City_ID": row["City_ID"], # replace city name with ID to reduce redundancy
                     "Population": int(city_population_clean),
                     "Timestamp": today
                    })

population_data = pd.DataFrame(population_data)
population_data

,City_ID,Population,Timestamp
0,1,3878100,2024-10-30
1,2,1964021,2024-10-30
2,3,1510378,2024-10-30


Now we'll add a new table definition to our SQL script to give this population data a home.

If you use the same script, be sure not to run the whole thing or you'll drop your schema (and with it your stored data) in the process.

```sql
CREATE TABLE populations (
	`City_ID` INT UNSIGNED NOT NULL,
    `Population` INT UNSIGNED,
    `Timestamp` DATE,
    FOREIGN KEY (`City_ID`) REFERENCES cities(`City_ID`)
    );
    
```

In [ ]:
population_data.to_sql("populations",
                      if_exists="append",
                      con=connection_string,
                      index=False)

3

Once again, a 3 indicates success.